In [ ]:
import pickle
with open('list_of_match_ids_for_training.pkl', 'rb') as file:
    match_ids = pickle.load(file)
print(len(match_ids), match_ids)

In [ ]:
import requests

def get_champion_data(api_key):
    url = f"https://ddragon.leagueoflegends.com/cdn/12.8.1/data/en_US/champion.json"
    response = requests.get(url)
    return response.json()

# Example usage
api_key = 'RGAPI-73de7a1d-bd48-401f-9ea2-12e33bd637f2'  # Replace with your valid API key
champion_data = get_champion_data(api_key)

# Create a dictionary mapping champion IDs to their names and classes
champion_classes = {
    "Tank": [],
    "Mage": [],
    "Assassin": [],
    "Marksman": [],
    "Support": [],
    "Fighter": []
}

for champ in champion_data['data'].values():
    # Assume champ['tags'] contains the class information
    for role in champ['tags']:
        if role in champion_classes:
            champion_classes[role].append(champ['name'])

print(champion_classes)
print(champion_classes.keys())

In [3]:
def kda(details):
    data = {}
    team_blue = []
    team_red = []
    team_blue_summoner_ids = []
    team_red_summoner_ids = []

    # Loop through participants to separate them into blue and red teams
    for i in range(10):
        if details['info']['participants'][i]['teamId'] == 100:
            team_blue.append(i)
        else:
            team_red.append(i)

    if details['info']['participants'][team_blue[0]]['win'] == True:
        data['blue_win'] = 1
    else:
        data['blue_win'] = 0

    # Initialize variables for blue and red teams
    blue_k = 0
    blue_a = 0
    blue_d = 0
    red_k = 0
    red_a = 0
    red_d = 0

    # Calculate stats for blue team
    for i in team_blue:
        blue_k += details['info']['participants'][i]['kills']
        blue_d += details['info']['participants'][i]['deaths']
        blue_a += details['info']['participants'][i]['assists']
        team_blue_summoner_ids.append(details['info']['participants'][i]['summonerId'])

    # Calculate stats for red team
    for i in team_red:
        red_k += details['info']['participants'][i]['kills']
        red_d += details['info']['participants'][i]['deaths']
        red_a += details['info']['participants'][i]['assists']
        team_red_summoner_ids.append(details['info']['participants'][i]['summonerId'])

    # Calculate KDA (Kills, Deaths, Assists) for each team
    blue_kda = (blue_k + blue_a) / max(1, blue_d)  # Avoid division by zero
    red_kda = (red_k + red_a) / max(1, red_d)  # Avoid division by zero

    # Store data into the dictionary
    data['blue_kills'] = blue_k
    data['blue_deaths'] = blue_d
    data['blue_assists'] = blue_a
    data['blue_kda'] = blue_kda

    data['red_kills'] = red_k
    data['red_deaths'] = red_d
    data['red_assists'] = red_a
    data['red_kda'] = red_kda

    return data, team_blue, team_red, team_blue_summoner_ids, team_red_summoner_ids

In [4]:
def champ(details, data, team_blue, team_red):
    for key in champion_classes:
        data[f'blue_{key}'] = 0
        data[f'red_{key}'] = 0

    for i in team_blue:
        champion_name = details['info']['participants'][i]['championName']
        for key in champion_classes:
            if champion_name in champion_classes[key]:
                champ_lvl = details['info']['participants'][i]['champLevel']
                champ_exp = details['info']['participants'][i]['champExperience']
                data[f'blue_{key}'] += champ_lvl * champ_exp

    for i in team_red:
        champion_name = details['info']['participants'][i]['championName']
        for key in champion_classes:
            if champion_name in champion_classes[key]:
                champ_lvl = details['info']['participants'][i]['champLevel']
                champ_exp = details['info']['participants'][i]['champExperience']
                data[f'red_{key}'] += champ_lvl * champ_exp

    return data


In [5]:
def rank_score(data, team_blue_summoner_ids, team_red_summoner_ids):
    avg_rank_score_red = 0
    avg_rank_score_blue = 0
    r = 0
    b = 0

    for summoner_id in team_blue_summoner_ids:
        rank_score = riot_api.get_combined_rank_score(summoner_id)
        if rank_score:
            avg_rank_score_blue += rank_score
            b += 1
        # print(f"Player Blue: Summoner ID: {summoner_id}, Combined Rank Score: {rank_score}")

    for summoner_id in team_red_summoner_ids:
        rank_score = riot_api.get_combined_rank_score(summoner_id)
        if rank_score:
            avg_rank_score_red += rank_score
            r += 1
        # print(f"Player Red: Summoner ID: {summoner_id}, Combined Rank Score: {rank_score}")

    avg_rank_score_red = avg_rank_score_red / r
    avg_rank_score_blue = avg_rank_score_blue / b
    data['rank_score_red'] = avg_rank_score_red
    data['rank_score_blue'] = avg_rank_score_blue
    return data

In [ ]:
import requests
import time
import pandas as pd

class RiotApi(object):
    def __init__(self, api_key: str, region="americas"):
        self.RIOT_API_KEY = api_key
        self.HEADER = {'X-Riot-Token': self.RIOT_API_KEY}
        self.REGION = region
        self.BASE_URL = ".api.riotgames.com/"

    def match_details(self, match_id):
        self.REGION = "americas"
        url = f"https://{self.REGION}{self.BASE_URL}/lol/match/v5/matches/{match_id}"
        request = requests.get(url, headers=self.HEADER)
        if request.status_code == 200:
            return request.json()
        else:
            print(f"Error: Received status code {request.status_code} for match details request. Response: {request.text}")
            return {}
    
    def get_rank_and_tier(self, summoner_id: str) -> dict:
        """Get Rank and Tier by Summoner ID
        @param summoner_id: Summoner ID
        @return: json object of rank and tier data
        """
        self.REGION = "NA1"
        url = f"https://{self.REGION}{self.BASE_URL}lol/league/v4/entries/by-summoner/{summoner_id}"
        request = requests.get(url, headers=self.HEADER)
        if request.status_code == 200:
            return request.json()
        else:
            print(f"Error: Received status code {request.status_code} for rank request. Response: {request.text}")
            return {}

    def get_rank_score(self, tier, rank):
        tier_scores = {
            'IRON': 1,
            'BRONZE': 2,
            'SILVER': 3,
            'GOLD': 4,
            'PLATINUM': 5,
            'EMERALD': 6,
            'DIAMOND': 7,
            'MASTER': 8,
            'GRANDMASTER': 9,
            'CHALLENGER': 10
        }

        rank_scores = {
            'IV': 1,
            'III': 2,
            'II': 3,
            'I': 4
        }
        
        if tier in ['MASTER', 'GRANDMASTER', 'CHALLENGER']:
            return tier_scores[tier] * 4  # Using a high score for these ranks
        else:
            return (tier_scores[tier] * 4) + rank_scores[rank]
    
    def get_combined_rank_score(self, summoner_id):
        rank_data = self.get_rank_and_tier(summoner_id)
        for entry in rank_data:
            if entry['queueType'] == 'RANKED_SOLO_5x5':
                tier = entry['tier']
                rank = entry['rank']
                return self.get_rank_score(tier, rank)
        return None  # If no rank found

# Example usage
API_KEY = 'RGAPI-488c2dd3-e42a-479e-92d6-5e8e35ba62b5'  # Replace with your valid API key

# Initialize the Riot API class
riot_api = RiotApi(API_KEY)

# Get match details for the first match ID
for i in range(len(match_ids)):
    match_id = match_ids[i]
    details = riot_api.match_details(match_id)
    print(len(details))
    data, team_blue, team_red, team_blue_summoner_ids, team_red_summoner_ids = kda(details)
    data = champ(details, data, team_blue, team_red)
    data = rank_score(data, team_blue_summoner_ids, team_red_summoner_ids)

    # appending multiple match data dictionaries
    matches_data.append(data)
    print(len(matches_data), "hi", i)


In [ ]:
print(matches_data)
print(len(matches_data))

In [14]:
import pickle
with open('list_of_match_details_for_training.pkl', 'wb') as file:
    pickle.dump(matches_data, file)

In [ ]:
with open('list_of_match_details_for_training.pkl', 'rb') as file:
    matches_data = pickle.load(file)
print(len(matches_data))

In [2]:
import os
import pandas as pd

matches_data = []
# Create the "data" folder if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')

# Create the DataFrame
train_df = pd.DataFrame(matches_data)

# Save the DataFrame to a CSV file in the "data" folder
train_df.to_csv('data/train_data_set.csv', index=False)